In [1]:
import os

In [55]:
%pwd

'c:\\Users\\Maza\\Desktop\\Pinecone_pipeline'

In [3]:
os.chdir("../")

In [56]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    read_data_dir: Path
    STATUS_FILE: str
    SCHEMA: dict



In [57]:
from vector_db_pipeline.constants import *
from vector_db_pipeline.utils.common import read_yaml, create_directories
from vector_db_pipeline import logger
import pandas as pd

In [58]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)
        

    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            read_data_dir=config.read_data_dir,
            STATUS_FILE=config.STATUS_FILE,
            SCHEMA=schema
        )

        return data_validation_config
    

In [59]:
import os
from vector_db_pipeline import logger
import pandas as pd

In [60]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config
        self.data = pd.read_json(self.config.read_data_dir, orient='records')


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            
            all_cols = list(self.data.columns)
            

            all_schema = self.config.SCHEMA
            all_schema['id'] = 'str'
            all_schema['values'] = 'list'
       

            
            for col in all_cols:
                if col not in all_schema.keys():
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'a') as f:
                        f.write(f"All columns present in data: {validation_status}\n")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'a') as f:
                        f.write(f"All columns present in data: {validation_status}\n")

            return logger.info(f"All columns present in data: {validation_status}")
        
        except Exception as e:
            raise e
        
    def validate_unique_index(self)-> bool:
        try:
            
            unique_id = len(self.data.id.unique()) == len(self.data)
            if unique_id:
             
                with open(self.config.STATUS_FILE, 'a') as f:
                    f.write(f"Unique ids : {unique_id}\n")
            else:
               
                with open(self.config.STATUS_FILE, 'a') as f:
                    f.write(f"Unique ids : {unique_id}\n")
            return logger.info(f"Unique ids : {unique_id}")
        
        except Exception as e:
            raise e
        

    def validate_column_types(self):
        """
        Get the types of values in each column of the DataFrame.

       
        Returns:
            dict: A dictionary where keys are column names and values are lists containing the types of values present in each column.
        """
        
        try:
            column_types = {}
            how_many_types = {}
            
            # Iterate over each column in the DataFrame
            for column in self.data.columns:
                # Create a set of unique types for values in the column
                types = set(type(value) for value in self.data[column])
                # Convert the set to a list for easier handling
                types_list = list(types)

                column_types[column] = types_list
                how_many_types[column] = len(types_list)
            
            

            # Check if elements in 'values' column are of type float
            vector_elemts_type = list(set(type(elem) for elem in self.data['values'][0]))
            if len(vector_elemts_type) != 1 or vector_elemts_type[0] != float:
                raise TypeError("Vector values are not of type float")

            # Check if all columns have a single type
            unique_types = all(value == 1 for value in how_many_types.values())

            # Check if 'id' column is of type string and 'values' column is of type list
            id_type = column_types['id'][0] == str
            vector_type = column_types['values'][0] == list

            if not id_type:
                raise TypeError("ID column is not of type string")
            elif not vector_type:
                raise TypeError("Values column is not of type list")
            elif not unique_types:
                raise TypeError("Several types present in data")

            # Log success message
            with open(self.config.STATUS_FILE, 'a') as f:
                f.write("Data types are correct\n")

            return logger.info("Data types are valid")
        except Exception as e:
            raise e


In [61]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_columns()
    data_validation.validate_unique_index()
    data_validation.get_column_types()
except Exception as e:
    raise e

[2024-04-15 10:48:35,683: INFO: common: yaml file: config\config.yaml loaded successfully:]
[2024-04-15 10:48:35,689: INFO: common: yaml file: schema.yaml loaded successfully:]
[2024-04-15 10:48:35,692: INFO: common: Directory already exists: artifacts/data_validation:]


{'date_scraped_timestamp': 'int', 'host': 'str', 'page_title': 'str', 'text': 'str', 'url': 'str', 'id': 'str', 'values': 'list'}
['id', 'values', 'text', 'host', 'page_title', 'chunk', 'url', 'timestamp']
[2024-04-15 10:48:36,011: INFO: 1103252934: All columns present in data: False:]
[2024-04-15 10:48:36,016: INFO: 1103252934: Unique ids : True:]
[2024-04-15 10:48:36,021: INFO: 1103252934: Data types are valid:]
